<a target="_blank" href="https://colab.research.google.com/github/cpacker/MemGPT/blob/main/memgpt/autogen/examples/memgpt_coder_autogen.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
%pip install pyautogen==0.2.0

In [ ]:
%pip install pymemgpt

In [ ]:
# You can get an OpenAI API key at https://platform.openai.com
OPENAI_API_KEY = "YOUR_API_KEY"

In [ ]:
"""Example of how to add MemGPT into an AutoGen groupchat

Based on the official AutoGen example here: https://github.com/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb
"""

import autogen
from memgpt.autogen.memgpt_agent import create_memgpt_autogen_agent_from_config


# This config is for AutoGen agents that are not powered by MemGPT
config_list = [
    {
        "model": "gpt-4",
        "api_key": OPENAI_API_KEY,
    },
]
llm_config = {"config_list": config_list, "seed": 42}


# This config is for AutoGen agents that powered by MemGPT
config_list_memgpt = [
    {
        "model": "gpt-4",
        "preset": "memgpt_chat",
        "model_wrapper": None,
        "model_endpoint_type": "openai",
        "model_endpoint": "https://api.openai.com/v1",
        "context_window": 8192,  # gpt-4 context window
        "openai_key": OPENAI_API_KEY,
    },
]
llm_config_memgpt = {"config_list": config_list_memgpt, "seed": 42}

In [ ]:
# The user agent
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE",  # needed?
    default_auto_reply="...",  # Set a default auto-reply message here
)

# The agent playing the role of the product manager (PM)
# Let's make this a non-MemGPT agent
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
    default_auto_reply="...",  # Set a default auto-reply message here
)

# If USE_MEMGPT is False, then this example will be the same as the official AutoGen repo (https://github.com/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb)
# If USE_MEMGPT is True, then we swap out the "coder" agent with a MemGPT agent
USE_MEMGPT = True

if not USE_MEMGPT:
    # In the AutoGen example, we create an AssistantAgent to play the role of the coder
    coder = autogen.AssistantAgent(
        name="Coder",
        llm_config=llm_config,
    )

else:
    # In our example, we swap this AutoGen agent with a MemGPT agent
    # This MemGPT agent will have all the benefits of MemGPT, ie persistent memory, etc.

    # We can use interface_kwargs to control what MemGPT outputs are seen by the groupchat
    interface_kwargs = {
        "debug": False,
        "show_inner_thoughts": True,
        "show_function_outputs": False,
    }

    coder = create_memgpt_autogen_agent_from_config(
        "MemGPT_coder",
        llm_config=llm_config_memgpt,
        system_message=f"I am a 10x engineer, trained in Python. I was the first engineer at Uber "
        f"(which I make sure to tell everyone I work with).\n"
        f"You are participating in a group chat with a user ({user_proxy.name}) "
        f"and a product manager ({pm.name}).",
        interface_kwargs=interface_kwargs,
        default_auto_reply="...",  # Set a default auto-reply message here (non-empty auto-reply is required for LM Studio)
    )

In [ ]:
# Initialize the group chat between the user and two LLM agents (PM and coder)
groupchat = autogen.GroupChat(agents=[user_proxy, pm, coder], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Begin the group chat with a message from the user
user_proxy.initiate_chat(
    manager,
    message="I want to design an app to make me one million dollars in one month. Yes, your heard that right.",
)